In [ ]:
# Lets import torch
import torch 
import torch.nn as nn
import torch.nn.functional as F

: 

: 

In [ ]:
torch.__version__

: 

In [ ]:
import pandas as pd
df = pd.read_csv('diabetes.csv')

: 

In [ ]:
df.head()

: 

In [ ]:
df.isna().sum().sum()

: 

In [ ]:
import numpy as np

: 

In [ ]:
df['Outcome'] = np.where(df['Outcome'] ==1, "Diabetic", "Non-Diabetic")

: 

In [ ]:
df.head()

: 

In [ ]:
import seaborn as sns
sns.pairplot(df, hue = "Outcome")

: 

In [ ]:
df.dtypes

: 

In [ ]:
df['Outcome'] = np.where(df['Outcome'] == "Diabetic", 1, 0)

: 

In [ ]:
df.dtypes

: 

In [ ]:
X = df.drop('Outcome', axis = 1).values
y = df['Outcome'].values

: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

: 

In [ ]:
# We have convert our independent features into float tensors and our dependent features not into float tensors, 
#so we can convert into long tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


: 

In [ ]:
class ANN_Model(nn.Module):
    def __init__(self,input_features= 8, hidden1=20, hidden2 = 20 ,out_features =2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    def forward(self,X):
        X = F.relu(self.f_connected1(X))
        X = F.relu(self.f_connected2(X))
        X=self.out(X)
        return X
torch.manual_seed(20)
model = ANN_Model()

: 

In [ ]:
model.parameters

: 

In [ ]:
# Backward Propagation
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

: 

In [ ]:
# Model training
epochs=500
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

: 

In [ ]:
# Prediction In X_test data
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

: 

In [ ]:
# Validation
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)
cm
#1st --> Actual class 0
#2nd --> Actual Class
#90 samples were correctly predicted as class 0.
#32 samples were correctly predicted as class 1.
#15 samples were falsely predicted as class 1 when they were actually class 0.
#17 samples were falsely predicted as class 0 when they were actually class 1.

: 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

: 

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

: 

In [ ]:
# Save the model
torch.save(model.state_dict(), 'diabetes_state_dict.pt')



: 

In [ ]:
# Save And Load the model
model=torch.load('diabetes_state_dict.pt')

: 

In [ ]:
# Model details
model.eval()

: 

In [ ]:
# New Data
lst1=[6.0, 130.0, 72.0, 40.0, 0.0, 25.6, 0.627, 45.0]

: 

In [ ]:
new_data=torch.tensor(lst1)

: 

In [ ]:
# Predict new data using Pytorch
with torch.no_grad():
    print(model(new_data))
    print(model(new_data).argmax().item())

: 